In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np

In [ ]:
#/kaggle/input/big-mart-sales-prediction/Submission.csv
#/kaggle/input/big-mart-sales-prediction/Train.csv
#/kaggle/input/big-mart-sales-prediction/Test.csv

## Problem Statement
The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and predict the sales of each product at a particular outlet.

Using this model, BigMart will try to understand the properties of products and outlets which play a key role in increasing sales.


# Data Cleaning

In [ ]:
df= pd.read_csv('/kaggle/input/big-mart-sales-prediction/Train.csv')
df1=pd.read_csv('/kaggle/input/big-mart-sales-prediction/Test.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Outlet_Size'].value_counts()

In [ ]:
df['Outlet_Location_Type'].value_counts()

In [ ]:
df['Outlet_Size'].value_counts()

In [ ]:
df.info()

In [ ]:
from statistics import mode
df['Item_Weight'].fillna(df['Item_Weight'].mean(),inplace=True)
df['Outlet_Size'].fillna(mode(df['Outlet_Size']),inplace=True)

In [ ]:
df.info() #format are right

In [ ]:
df['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':"Low Fat"},inplace=True) #check for outliers 
# and remove them

In [ ]:
df['Item_Fat_Content'].value_counts()

In [ ]:
sns.boxplot(df['Item_Weight'])

In [ ]:
sns.scatterplot(x='Item_Visibility',y='Item_Visibility',data=df)

In [ ]:
df['Item_Type'].value_counts()

In [ ]:
sns.boxplot(df['Item_MRP'])

In [ ]:
df['Outlet_Identifier'].value_counts()

In [ ]:
df['Outlet_Establishment_Year'].value_counts()

In [ ]:
df['Outlet_Location_Type'].value_counts()

In [ ]:
df['Outlet_Type'].value_counts()

In [ ]:
df1['Item_Weight'].fillna(df1['Item_Weight'].mean(),inplace=True)
df1['Outlet_Size'].fillna(mode(df1['Outlet_Size']),inplace=True)
df1['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':"Low Fat"},inplace=True) #check for outliers 


# Data Analysis


In [ ]:
sns.pairplot(df)

1. Item visibility and item sales outlet is left skewed is left skewed, we can use standard scalar for normalization

2. as we can see in the plot they are not seperated by linear regression we can try decision tree, random forest or boosting technique to split the and predict data(using information gain or gini index) or SVR

3. We can see that low sales has comparatively low visibility

4. High MRP has comparatively low sales

In [ ]:
sns.barplot(x='Item_Fat_Content',y='Item_Outlet_Sales',data=df)

Mean are almost the same for low fat and regular but regular has little more sales than lowfat

In [ ]:
plt.figure(figsize=(17,7))
sns.barplot(x='Item_Type',y='Item_Outlet_Sales',data=df)
plt.xticks(rotation=90)

 soft drinks, baking goods, and health and hygene has comparatively low sales 

Means of different item type are varying we can use it in our prediction

In [ ]:
for i in df[['Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']].columns:
    chart = sns.barplot(x=df[i], y=df['Item_Outlet_Sales'])
    chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
    plt.show()

Outlet_Establishment_Year, Outlet_Size, Outlet_Location_Type and Outlet_Type has a good varying values 
1. Grocery store has a really low sales
2. Teir has comparatively low sales 
3. 1998 we can see has low sales
4. small size has comparatively low sales

we can look into it as to why is they have low sales and try to increase their saless

In [ ]:
df.corr()

There is high correlation between mrp and sales we can clearly say that high mrp has low sales

there is very less correlation between outlet year and weight

# Data preprocessing

In [ ]:
df.iloc[:,1:-1].head()

In [ ]:
X_train=pd.get_dummies(df.iloc[:,1:-1])
X_train.head()

In [ ]:
y_train=pd.get_dummies(df1.iloc[:,1:])
y_train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
y_train = sc.fit_transform(y_train)
#X_test = sc.transform(X_test)

In [ ]:
X_test=df.iloc[:,-1].values

In [ ]:
import statsmodels.api as sm

X_sm  = sm.add_constant(X_train)
model = sm.OLS(X_test,X_sm)
model.fit().summary()

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import cross_val_score

lm = LinearRegression()
lm.fit(X_train, X_test)

np.mean(cross_val_score(lm,X_train,X_test, scoring = 'neg_mean_absolute_error', cv= 3))

# Lasso regression

In [ ]:
alpha = []
error = []

for i in range(1,200):
    alpha.append(i/1)
    lml = Lasso(alpha=(i/1))
    error.append(np.mean(cross_val_score(lml,X_train,X_test, scoring = 'neg_mean_absolute_error', cv= 3)))
    
plt.plot(alpha,error)

In [ ]:
err = tuple(zip(alpha,error))
df_err = pd.DataFrame(err, columns = ['alpha','error'])
df_err[df_err.error == max(df_err.error)]

In [ ]:
lm_l = Lasso(alpha=32.0)
lm_l.fit(X_train,X_test)
np.mean(cross_val_score(lm_l,X_train,X_test, scoring = 'neg_mean_absolute_error', cv= 3))

# Decision tree, Random forest and boosting technique

In [ ]:
from sklearn.tree import DecisionTreeRegressor
Regressor = DecisionTreeRegressor(random_state = 0)
Regressor.fit(X_train, X_test)
np.mean(cross_val_score(Regressor,X_train,X_test, scoring = 'neg_mean_absolute_error', cv= 3))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(bootstrap=True, max_features=0.9,
                           min_samples_leaf=18, min_samples_split=20, n_estimators=400,criterion='mse',max_depth=10.2,
                           max_samples=5665)
rf.fit(X_train, X_test)
np.mean(cross_val_score(rf,X_train,X_test,scoring = 'neg_mean_absolute_error', cv= 3))

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X_train, X_test)
np.mean(cross_val_score(model,X_train,X_test,scoring = 'neg_mean_absolute_error', cv= 3))

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
classifier = AdaBoostRegressor(
    n_estimators=500
)
classifier.fit(X_train, X_test)
np.mean(cross_val_score(classifier,X_train,X_test,scoring = 'neg_mean_absolute_error', cv= 3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1,1.01]

for learning_rate in lr_list:
    gb_clf = GradientBoostingRegressor(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, X_test)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, X_test)))

In [ ]:
gb_clf = GradientBoostingRegressor(n_estimators=200, learning_rate=1.01, max_features=2, max_depth=2, random_state=0)
gb_clf.fit(X_train, X_test)
np.mean(cross_val_score(gb_clf,X_train,X_test,scoring = 'neg_mean_absolute_error', cv= 3))

In [ ]:
import sklearn
extra1=sklearn.ensemble.ExtraTreesRegressor(bootstrap=True, max_features=0.8,
                    min_samples_leaf=18, min_samples_split=6, n_estimators=100)
extra1.fit(X_train,X_test)
np.mean(cross_val_score(extra1,X_train,X_test,scoring = 'neg_mean_absolute_error', cv= 3))

## As the accuracy of ExtraTreesRegressor fit the best we will use ExtraTreesRegressor 

### Used random searchcv and gridsearchcv for optimisation

In [ ]:
import sklearn
extra=sklearn.ensemble.ExtraTreesRegressor(bootstrap=True, max_features=0.9,
                    min_samples_leaf=18, min_samples_split=20, n_estimators=400,criterion='mse',max_depth=10.2,
                                           max_samples=5665)
extra.fit(X_train,X_test)
np.mean(cross_val_score(extra,X_train,X_test,scoring = 'neg_mean_absolute_error', cv= 3))

In [ ]:
#-756.0596882610718
#-755.66193353781
#-755.105940462904
e=extra.predict(y_train)
df1['Item_Outlet_Sales']=e
df1[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']].to_csv('samplesub2.csv',index=False)